In [1]:
import os
import re
import json
import subprocess

from openai import OpenAI
import sys
sys.path.append('../')
from prompts.design_prompt import design_prompt
from prompts.parsing_prompt import parsing_prompt
from prompts.scripting_prompt import scripting_prompt

In [2]:
with open('../config/config.json') as f:
     config = json.load(f)

In [3]:
OPENSCAD_EXEC = config["OPENSCAD_EXEC_PATH"]
FILE_DIR = "../common/helix_gear"
DESIGN_FILENAME = "gears_design.md"
PARAMETERS_FILENAME = "gears_parameters.json"
SCRIPT_FILENAME = "gears_script.scad"
if not os.path.exists(FILE_DIR):
    os.makedirs(FILE_DIR)

In [4]:
client = OpenAI(api_key=config['OPENAI_API_KEY'])

In [5]:
design_description = f""" 
Suppose there is a one-stage system with two helical gears with a 20 degrees helix angle.
Gear axes are parallel and the distance between two axes is 50mm. 
The height of driven and driving gears are 20mm and 15mm respectively, the pressure angle are both 20 degrees.
The desired gear ratio of the reduction system is 1.5:1, 
Set the driving gear to the center of the plane, which is the coordinate (0, 0)
Based on the given information, computer module number, teeth number of both gears and the coordniate of the driven gear centers.
"""

In [6]:
design_messages=[
        {"role": "system", "content": design_prompt},
        {"role": "user", "content": design_description},
    ]

In [7]:
# stop_tokens = ['']
# collected_tokens = []

# res = client.chat.completions.create(
#     model=config['GPT_MODEL'],
#     messages=design_messages,
#     stream=True,
# )

# for chunk in res:
#     temp_tokens = chunk.choices[0].delta.content
#     if temp_tokens not in stop_tokens:
#         collected_tokens.append(temp_tokens)
#     else:
#         chunk_res = "".join(collected_tokens).strip()
#         print(chunk_res)
#         collected_tokens = []

res = client.chat.completions.create(
    model=config['GPT_MODEL'],
    messages=design_messages,
    temperature=0.1
)

In [8]:
print(res.choices[0].message.content)

To design a gear system with the given requirements, we need to calculate the module, number of teeth, and the coordinates of the driven gear center. Let's start by defining the gear ratio and the relationship between the gears.

### Gear Ratio
The gear ratio (i) is defined as the ratio of the number of teeth on the driven gear (N2) to the number of teeth on the driving gear (N1):

$$ i = \frac{N2}{N1} $$

Given that the desired gear ratio is 1.5:1, we can express N2 in terms of N1:

$$ N2 = 1.5 \cdot N1 $$

### Module Calculation
The module (m) is a measure of the size of the teeth and is related to the pitch diameter (d) and the number of teeth (N) by the formula:

$$ m = \frac{d}{N} $$

Since the axes of the two gears are parallel and the center distance (a) is 50mm, we can relate the pitch diameters of the two gears to the center distance:

$$ a = \frac{d1 + d2}{2} $$

Substituting the relationship between d and N, and knowing that d2 = 1.5 * d1, we get:

$$ 50 = \frac{d1 + 1.5 \cd

In [9]:
with open(os.path.join(FILE_DIR, DESIGN_FILENAME), "w") as f:
    f.write(res.choices[0].message.content)

In [22]:
parsing_description = \
f"gear 1 (driving gear): helical gear, module 2.5mm, teeth 16, pitch diameter 40mm, helix angle 20 degrees, height 15mm, coordinate (0, 0) " + \
f"pressure angle 20 degrees" + \
f"gear 2 (driven gear): helical gear, module 2.5mm, teeth 24, pitch diameter 60mm, helix angle 20 degrees, height 20mm, coordinate (50, 0) " + \
f"pressure angle 20 degrees"

In [23]:
parsing_messages = [
    {"role": "system", "content": parsing_prompt},
    {"role": "user", "content": parsing_description},
]

In [24]:
parsing_response = client.chat.completions.create(
    model=config['GPT_MODEL'],
    response_format={ "type": "json_object" },
    messages=parsing_messages,
)

In [25]:
gears_parameters = parsing_response.choices[0].message.content.strip()
print(gears_parameters)

{
   "gear 1": {
      "gear_type": "helix",
      "source": "driving",
      "coordinate": {
         "x": 0,
         "y": 0,
         "z": 0,
         "alpha": 0,
         "beta": 0,
         "gamma": 0
      },
      "unit": "mm",
      "module": 2.5,
      "teeth": 16,
      "height": 15,
      "pitch_d": 40,
      "helix_angle": 20,
      "pitch_angle": 0,
      "pressure_angle": 20
   },
   "gear 2": {
      "gear_type": "helix",
      "source": "driven",
      "coordinate": {
         "x": 50,
         "y": 0,
         "z": 0,
         "alpha": 0,
         "beta": 0,
         "gamma": 0
      },
      "unit": "mm",
      "module": 2.5,
      "teeth": 24,
      "height": 20,
      "pitch_d": 60,
      "helix_angle": 20,
      "pitch_angle": 0,
      "pressure_angle": 20
   }
}


In [26]:
with open(os.path.join(FILE_DIR, PARAMETERS_FILENAME), "w") as f:
    f.write(gears_parameters)

In [27]:
with open(os.path.join(FILE_DIR, PARAMETERS_FILENAME), "r") as f:
    parameters = json.load(f)
parameters = str(parameters)

scripting_description = f"Write a script according to provided parameters of gears \n" + parameters

In [28]:
script_messages = [
    {"role": "system", "content": scripting_prompt},
    {"role": "user", "content": scripting_description}
]

In [29]:
scripting_response = client.chat.completions.create(
    model=config['GPT_MODEL'],
    messages=script_messages,
    temperature=0
)

In [30]:
def script_postprocessing(script):
    # To remove textual perfix or suffix information in the raw script generated by GPT-4
    pattern = re.compile(r'```(?:openscad|scad)(.*?)```', re.DOTALL)
    matches = re.findall(pattern, script)
    clean_script = ""
    if len(matches) > 0:
        for m in matches:
            clean_script = clean_script + m.strip()
    else:
        clean_script = script
    return clean_script

In [31]:
script = scripting_response.choices[0].message.content.strip()
script = script_postprocessing(script)
print(script)

translate([0,0,0]) rotate([0, 0, 0])
gear_helix(m=2.5, z=16, h=15, w=20, w_helix=20);

translate([50,0,0]) rotate([0, 0, 360 / (24 * 2)])
gear_helix(m=2.5, z=24, h=20, w=20, w_helix=-20);


In [32]:
with open(os.path.join(FILE_DIR, SCRIPT_FILENAME), "w") as f:
    f.write("include <gears.scad>")

with open(os.path.join(FILE_DIR, SCRIPT_FILENAME), "a") as f:
    f.write("\n" + script)

In [33]:
subprocess.Popen([OPENSCAD_EXEC, os.path.join(FILE_DIR, SCRIPT_FILENAME)])

<Popen: returncode: None args: ['D:\\openSCAD\\install\\openscad.exe', '../c...>